In [ ]:
%matplotlib inline
from astropy.table import Table
from matplotlib import pyplot as plt
from skimage import io
import numpy as np
from IPython import display

In [ ]:
results = Table.read('./results_table_full_0.5_2.0.fits')

In [ ]:
results_df = results.to_pandas()

In [ ]:
dup_mask = results_df[['NAME', 'RA_detc']].duplicated('NAME', keep=False)

In [ ]:
dup_cat = results_df[dup_mask]

In [ ]:
dup_cat

In [ ]:
dup_cat = Table.from_pandas(dup_cat)
dup_cat.filled(-1.0).write('results_table_dups_0.5_2.0.fits', overwrite=True)

In [ ]:
dup_mask.values

# Now we are gonna visually inspect the duplicate fields

This is gonna let us identify which is the "primary" source

In [ ]:
# make a catalog mask

fig = plt.figure(1, figsize=(10, 10))
ax = fig.add_subplot()

plt.ion()

i = 0
name_old= ''
for name, index, dup in zip(results['NAME'], results['INDEX'], dup_mask):
    if not dup:
        i += 1
        continue

    try:
        name = name.replace(' ', '_')

        if name == name_old:
            pass
        else:
            display.clear_output(wait=True)
            jpgfile = f'../data_full_new/{name}/{name}_XRT_vtp_zoom.png'
            jpg_array = io.imread(jpgfile)
            name_old = name
            
            ax.imshow(jpg_array, origin='upper', interpolation='bicubic')
            ax.set_title(f'{name}')

            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.yaxis.set_ticks([])
            ax.xaxis.set_ticks([])

            plt.show()
            display.display(fig)
            
    except NameError:
        display.clear_output(wait=True)
        jpgfile = f'../data_full_new/{name}/{name}_XRT_vtp_zoom.png'
        jpg_array = io.imread(jpgfile)
        name_old = name
        
        ax.imshow(jpg_array, origin='upper', interpolation='bicubic')
        ax.set_title(f'{name}')

        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.yaxis.set_ticks([])
        ax.xaxis.set_ticks([])

        plt.show()
        display.display(fig)

    x = input(f"Is {index} the main source? [n]y.")
    
    if x == 'y':
        print('updated')
        dup_mask[i] = False
    
    i += 1

# Save the results!

So we don't have to look at all of them again. 

Uncomment the next cell if you are gonna redo the inspections!

In [ ]:
# np.save('vis_inspects_mask', dup_mask)

In [ ]:
inspect_cat = results[~dup_mask.values]

In [ ]:
inspect_cat.filled(-1.0).write('results_table_inspected_0.5_2.0.fits', overwrite=True)